In [16]:
#need pip install spotipy to run this
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import base64
from requests import post, get
from config import cid, secret

In [17]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [18]:
results = sp.search(q='artist:' + 'Drake', type='artist')
print(results)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3ADrake&type=artist&offset=0&limit=10', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'}, 'followers': {'href': None, 'total': 73867865}, 'genres': ['canadian hip hop', 'canadian pop', 'hip hop', 'pop', 'rap', 'toronto rap'], 'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4', 'id': '3TVXtAsR1Inumwj472S9r4', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eb4293385d324db8558179afd9', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab676161000051744293385d324db8558179afd9', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f1784293385d324db8558179afd9', 'width': 160}], 'name': 'Drake', 'popularity': 97, 'type': 'artist', 'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/0p4ViyfJUTW0IT4SCBLexf'}, 'followers': {'href': None, 'total

In [13]:
auth_string = cid + ":" + secret
auth_bytes = auth_string.encode('utf-8')
auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')
url = "https://accounts.spotify.com/api/token"
headers = {
    'Authorization': 'Basic ' + auth_base64,
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {"grant_type": "client_credentials"}
result = post(url, headers=headers, data=data)
json_result = json.loads(result.content)
token = json_result['access_token']
print(token)

BQDydFFormshibOzPKU8Bljiy3blksqVRlCBZkT9IOH9GJtyS6YFufp8wBtMdM3XBzLFG89hA2DzfZBR7DqY22mjIf7pIgy1btA7R4UYWt07N3REyOeH


In [14]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX7Jl5KP2eZaS?si=fa88101bc2b04d96"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [20]:
sp.playlist_tracks(playlist_URI)['items'][0]['track']['duration_ms']

200040

In [23]:
genres = ''
song_name = []
artist_name = []
artist_pop = []
genre_list = []
album = []
track_pop = []
song_duration = []
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    #print(track_uri)
    #track = sp.track(track_uri)
    #print(track)
    #break
    song_genre = sp.track(track_uri)
    #print(song_genre['genre'])
    
    #Track name
    song_name.append(track["track"]["name"])
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    #print(f"Uri:{artist_uri}")
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name.append(track["track"]["artists"][0]["name"])
    artist_pop.append(artist_info["popularity"])
    artist_genres = artist_info["genres"]
    for genre in artist_genres:
        if genre == artist_genres[-1]:
            genres = genres + genre + ' '
        else: 
            genres = genres + genre + ', '
    genre_list.append(genres)
    genres = ''
    #Album
    album.append(track["track"]["album"]["name"])
    
    song_duration.append((track['track']['duration_ms']) / 60000)
    
    #Popularity of the track
    track_pop.append(track["track"]["popularity"])
top_fifty_df = pd.DataFrame({"Song Name": song_name,
                             "Song Duration(min)": song_duration,
                            "Artist Name": artist_name,
                            "Artist Popularity": artist_pop,
                            "Song Genre": genre_list,
                            "Album": album,
                            "Song Popularity": track_pop})
top_fifty_df

,Song Name,Song Duration(min),Artist Name,Artist Popularity,Song Genre,Album,Song Popularity
0,Blinding Lights,3.334000,The Weeknd,97,"canadian contemporary r&b, canadian pop, pop",After Hours,92
1,Dance Monkey,3.490633,Tones And I,72,australian pop,Welcome To The Madhouse (Deluxe),76
2,The Box,3.277533,Roddy Ricch,81,"melodic rap, rap, trap",Please Excuse Me for Being Antisocial,83
3,Roses - Imanbek Remix,2.936967,SAINt JHN,71,"melodic rap, slap house",Roses (Imanbek Remix),59
4,Don't Start Now,3.054833,Dua Lipa,88,"dance pop, pop, uk pop",Future Nostalgia,83
5,ROCKSTAR (feat. Roddy Ricch),3.028883,DaBaby,82,"hip hop, north carolina hip hop, rap",BLAME IT ON BABY,79
6,Watermelon Sugar,2.900000,Harry Styles,90,pop,Fine Line,89
7,death bed (coffee for your head),2.888883,Powfu,71,"sad lo-fi, sad rap",death bed (coffee for your head),83
8,Falling,2.656350,Trevor Daniel,67,melodic rap,Nicotine,79
9,Someone You Loved,3.036000,Lewis Capaldi,83,"pop, uk pop",Divinely Uninspired To A Hellish Extent,89


In [80]:
artist_name = 'Drake'
url = 'https://api.spotify.com/v1/search'
headers = {"Authorization": "Bearer " + token}
query = f"?q={artist_name}&type=artist&limit=1"
query_url = url + query
result = get(query_url, headers=headers)
json_result = json.loads(result.content)
json_result['artists']

{'href': 'https://api.spotify.com/v1/search?query=Drake&type=artist&offset=0&limit=1',
 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
   'followers': {'href': None, 'total': 73834874},
   'genres': ['canadian hip hop',
    'canadian pop',
    'hip hop',
    'rap',
    'toronto rap'],
   'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
   'id': '3TVXtAsR1Inumwj472S9r4',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5eb4293385d324db8558179afd9',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab676161000051744293385d324db8558179afd9',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f1784293385d324db8558179afd9',
     'width': 160}],
   'name': 'Drake',
   'popularity': 97,
   'type': 'artist',
   'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
 'limit': 1,
 'next': 'https://api.spotify.com/v1/search?query=Drake&

In [88]:
artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,10000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'year:2018', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': None} returned 400 due to Bad request.


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=year%3A2018&limit=50&offset=1000&type=track:
 Bad request., reason: None